# 원, 사각형, 삼각형 구분하기

## 3. Data Augmentation을 사용한 CNN 모델

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 부풀리기

In [3]:
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

In [4]:
# 데이터 부풀리기를 적용한 훈련 데이터 제너레이터
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=[0.8, 2.0],
                                   horizontal_flip=True,
                                   vertical_flip=True)

In [5]:
!mkdir preview

In [6]:
img = load_img('hard_handwriting_shape/train/triangle/triangle001.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0

for batch in train_datagen.flow(x, batch_size=1, save_to_dir='preview',
                                save_prefix='tri', save_format='png'):
    i += 1
    if i > 30:
        break

In [7]:
!ls preview

tri_0_1270.png	tri_0_3759.png	tri_0_5798.png	tri_0_7897.png	tri_0_9797.png
tri_0_1302.png	tri_0_3825.png	tri_0_5881.png	tri_0_8211.png	tri_0_9854.png
tri_0_1585.png	tri_0_403.png	tri_0_6416.png	tri_0_8288.png	tri_0_9972.png
tri_0_2407.png	tri_0_4949.png	tri_0_6427.png	tri_0_8923.png
tri_0_2997.png	tri_0_5158.png	tri_0_6646.png	tri_0_9089.png
tri_0_3418.png	tri_0_519.png	tri_0_7259.png	tri_0_928.png
tri_0_3503.png	tri_0_5672.png	tri_0_7282.png	tri_0_9575.png


### 데이터셋 생성하기

In [8]:
train_generator = train_datagen.flow_from_directory(
    'hard_handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 45 images belonging to 3 classes.


In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'hard_handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode='categorical'
)

Found 15 images belonging to 3 classes.


### 모델 정의/설정

In [10]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(24,24,3)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')             
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               819328    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
__________________________________________________

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 모델 학습
- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용할 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정.

In [12]:
#model.fit_generator(       - deprecated
model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=200,
    validation_data=test_generator,
    validation_steps=5
)

Epoch 1/200
15/15 [==============================] - 1s 58ms/step - loss: 1.4285 - accuracy: 0.3223 - val_loss: 1.1030 - val_accuracy: 0.4667
Epoch 2/200
15/15 [==============================] - 0s 21ms/step - loss: 1.1157 - accuracy: 0.2729 - val_loss: 1.0764 - val_accuracy: 0.5333
Epoch 3/200
15/15 [==============================] - 0s 24ms/step - loss: 1.0337 - accuracy: 0.5268 - val_loss: 1.0379 - val_accuracy: 0.4667
Epoch 4/200
15/15 [==============================] - 0s 22ms/step - loss: 0.8743 - accuracy: 0.6928 - val_loss: 1.1611 - val_accuracy: 0.4000
Epoch 5/200
15/15 [==============================] - 0s 20ms/step - loss: 0.7712 - accuracy: 0.6748 - val_loss: 1.0642 - val_accuracy: 0.6000
Epoch 6/200
15/15 [==============================] - 0s 22ms/step - loss: 0.4931 - accuracy: 0.9290 - val_loss: 1.2742 - val_accuracy: 0.4667
Epoch 7/200
15/15 [==============================] - 0s 21ms/step - loss: 0.5729 - accuracy: 0.7912 - val_loss: 1.2998 - val_accuracy: 0.6000
Epoch 

### 모델 평가

In [13]:
# acc = model.evaluate_generator(test_generator, steps=5) - deprecated
acc = model.evaluate(test_generator, steps=5)
print(f'Accuracy: {acc[1]:.4f}')

5/5 [==============================] - 0s 6ms/step - loss: 3.4258 - accuracy: 0.8000
Accuracy: 0.8000
